In [1]:
from copyreg import pickle
!python -V

Python 3.10.4


MLflow setup:

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///mlflow.db --default-artifact-root ./mlruns --host 0.0.0.0 --port 5000`

Import libraries

In [2]:
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

import mlflow

import pandas as pd
import numpy as np
from toolz import compose
import pickle

from sklearn.model_selection import train_test_split

from pandas.core.common import SettingWithCopyWarning
from matplotlib_inline import backend_inline

import warnings

backend_inline.set_matplotlib_formats('svg')
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [3]:
import os

_data_root = os.path.join("./../", 'data')
_data_root_raw = os.path.join(_data_root, 'raw')
_data_root_processed =  os.path.join(_data_root, 'processed')

_train_dirpath = os.path.join(_data_root_raw, "train")
_train_filepath = os.path.join(_train_dirpath, "train.csv")
_test_dirpath = os.path.join(_data_root_raw, "test")
_test_filepath = os.path.join(_test_dirpath, "test.csv")

_train_processed_dirpath = os.path.join(_data_root_processed, "train")
_train_processed_filepath = os.path.join(_data_root_processed, "train.csv")
_valid_processed_dirpath = os.path.join(_data_root_processed, "valid")
_valid_processed_filepath = os.path.join(_data_root_processed, "valid.csv")
_test_processed_dirpath = os.path.join(_data_root_processed, "test")
_test_processed_filepath = os.path.join(_data_root_processed, "test.csv")

os.makedirs(_train_dirpath, exist_ok=True)
os.makedirs(_test_dirpath, exist_ok=True)
os.makedirs(_train_processed_dirpath, exist_ok=True)
os.makedirs(_valid_processed_dirpath, exist_ok=True)
os.makedirs(_test_processed_dirpath, exist_ok=True)


## Download data from kaggle, unzip it and copy it to data folder


In [5]:
def download_data():
    !kaggle competitions download -c titanic -p {_data_root} --force
    !unzip -o {_data_root}/"titanic.zip" -d {_data_root}
    !cp {_data_root}/"train.csv" {_train_filepath}
    !cp {_data_root}/"test.csv" {_test_filepath}

    # clean up
    !rm  {_data_root}/*.csv  {_data_root}/*.zip

def extract_target(data: pd.DataFrame, target="Survived"):
    targets = data[target].values
    return targets

def preprocess_df(df: pd.DataFrame, transforms, categorical, numerical):
    """Return processed features dict and target."""

    # Apply in-between transformations
    df = compose(*transforms[::-1])(df)

    # For dict vectorizer: int = ignored, str = one-hot
    df[categorical] = df[categorical].astype(str)

    # Convert dataframe to feature dictionaries
    feature_dicts = df[categorical + numerical].to_dict(orient='records')

    return feature_dicts


def read_data(filename):
    """Return processed features dict and target."""

    # Load dataset
    if filename.endswith('parquet'):
        df = pd.read_parquet(filename)
    elif filename.endswith('csv'):
        df = pd.read_csv(filename)
    else:
        raise "Error: not supported file format."

    return df

def preprocess(df: pd.DataFrame, dv: DictVectorizer, fit_dv: bool = False):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    if fit_dv:
        X = dv.fit_transform(dicts)
    else:
        X = dv.transform(dicts)
    return X, dv

def preprocess_no_extract_target(filename, transforms, categorical, numerical):
    df = read_data(filename)

    feature_dicts = preprocess_df(df, transforms, categorical, numerical)

    return feature_dicts

def split_train_read(filename: str, val_size=0.2, random_state=42):
    df_train_full = read_data(filename)

    df_train, df_val = train_test_split(df_train_full, test_size=val_size, random_state=random_state)
    return df_train, df_val

def save_preprocessed(df: pd.DataFrame, path):
    df.to_csv(path)

def dump_pickle(obj, filename):
    with open(filename, "wb") as f_out:
        return pickle.dump(obj, f_out)

def run():
    mlflow.set_tracking_uri("http://127.0.0.1:5000")
    mlflow.set_experiment("titanic-experiment")
    mlflow.autolog()
    with mlflow.start_run():
        download_data()
        transforms = []
        target = 'Survived'
        categorical = ['Sex', 'Pclass', 'Embarked', 'SibSp', 'Parch']
        numerical = ['Fare']

        df_train, df_val = split_train_read(_train_filepath, val_size=0.2, random_state=42)

        train_dicts, y_train = preprocess_df(df_train, transforms, categorical, numerical), extract_target(df_train)
        val_dicts, y_val = preprocess_df(df_val, transforms, categorical, numerical), extract_target(df_val)

        df_test = read_data(_test_filepath)
        test_dicts = preprocess_df(df_test, transforms, categorical, numerical)

        # Fit all possible categories
        dv = DictVectorizer()
        dv.fit(train_dicts)

        X_train = dv.transform(train_dicts)
        X_val = dv.transform(val_dicts)
        X_test = dv.transform(test_dicts)

        model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)

        model.fit(X_train, y_train)
        dump_pickle(model, os.path.join("../models", "rfc.pkl"))
        y_pred = model.predict(X_val)

        accuracy = np.round(accuracy_score(y_val, y_pred), 4)
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_artifact(local_path=os.path.join("../models", "rfc.pkl"), artifact_path="models_pickle")
        print(accuracy)


In [6]:
run()

2022/07/22 20:07:03 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.


  0%|                                               | 0.00/34.1k [00:00<?, ?B/s]
100%|██████████████████████████████████████| 34.1k/34.1k [00:00<00:00, 1.91MB/s]
Archive:  ./../data/titanic.zip
  inflating: ./../data/gender_submission.csv  
  inflating: ./../data/test.csv      
  inflating: ./../data/train.csv     


2022/07/22 20:07:10 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/michal/.local/share/virtualenvs/kaggle-titanic-experiments-_bPcWiX9/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."


0.8045
